In [1]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

os.environ['OPENAI_API_KEY']="enter_openai_api_key"

In [ ]:
pdf=PyPDFLoader("file.pdf") #file location in device
page=pdf.load()
#print(len(page)) 
split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
chunk=split.split_documents(page)
#for i, ch in enumerate(chunk): 
 #   print(f"Chunk{i}:\n")
  #  print(ch.page_content)
   # print("----------")
embeddings=OpenAIEmbeddings()
vect=FAISS.from_documents(chunk,embeddings) 

In [8]:
memory=ConversationBufferMemory(memory_key="history",return_messages=True)
prompt_template="""
you are an AI assistant with access to the document, you can also have a general conversation with the user in the language the user prefers : {content}
you are also in an ongoing conversation and here is the context of the conversation so far: {history}
respond to the following user question in a helpful and informative manner: {inp}
"""
prompt=PromptTemplate(
    input_variables=["content","history","inp"],
    template=prompt_template,
)

llm=OpenAI(temperature=0.7)

def cbot(q):
    s=vect.similarity_search(q)
    c="\n\n".join([doc.page_content for doc in s])
    f=prompt.format(
        content=c,
        history=memory.load_memory_variables({})["history"],
        inp=q
    )
    response=llm(f)
    memory.save_context({"input":q},{"output":response})
    return response

if __name__=="__main__":
    while True:
        inp=input("\nuser: ")
        if inp=='exit':
            break
        b=cbot(inp)
        print(f"chatbot: {b}")


user:  my name is tushar


chatbot: 
Hello Tushar, it's nice to meet you! I am an AI assistant designed to assist with tasks and have conversations in different languages. Is there something specific you would like to know or discuss?



user:  is the name tushar mentioned in the document


chatbot: 
Yes, the name Tushar is mentioned in the document. Tushar Sharma is listed as one of the authors and his email address is also included in the document. Is there anything else you would like to know about the document or Tushar?



user:  what is my name


chatbot: 
Your name is Tushar, as mentioned in our previous conversation. Is there anything else you would like to know?



user:  exit
